<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Intro" data-toc-modified-id="Intro-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro</a></span></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#Face-Alignment" data-toc-modified-id="Face-Alignment-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Face Alignment</a></span><ul class="toc-item"><li><span><a href="#Landmark-Detection" data-toc-modified-id="Landmark-Detection-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Landmark Detection</a></span></li><li><span><a href="#Find-Convex-Hull" data-toc-modified-id="Find-Convex-Hull-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Find Convex Hull</a></span></li><li><span><a href="#Delaunay-Triangulation" data-toc-modified-id="Delaunay-Triangulation-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Delaunay Triangulation</a></span><ul class="toc-item"><li><span><a href="#Plot-Triangulation" data-toc-modified-id="Plot-Triangulation-3.3.1"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>Plot Triangulation</a></span></li></ul></li><li><span><a href="#Affine-warp-triangles" data-toc-modified-id="Affine-warp-triangles-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Affine warp triangles</a></span></li></ul></li><li><span><a href="#Seamless-Cloning" data-toc-modified-id="Seamless-Cloning-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Seamless Cloning</a></span></li></ul></div>

# Intro
Notebook exploring face-swap in Python.

Resources:
* [Face Swap using OpenCV](https://www.learnopencv.com/face-swap-using-opencv-c-python/)
* [How to install dlib](https://www.pyimagesearch.com/2017/03/27/how-to-install-dlib/)
* [Detect eyes, nose, lips, and jaw with dlib, OpenCV, and Python](https://www.pyimagesearch.com/2017/04/10/detect-eyes-nose-lips-jaw-dlib-opencv-python/)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os
from os.path import join
import sys

import dlib
import cv2

# Plotting
#%matplotlib notebook
%matplotlib inline

sns.set_context("paper")
sns.set_style("dark")

from face_swap import utils

%load_ext autoreload
%autoreload 2

In [ ]:
data_folder = "/datasets/"

# Load Data

In [ ]:
# Load two random celeba faces
from_face = cv2.cvtColor(cv2.imread(join(data_folder, "img_align_celeba", 
                            "000{}{}{}.jpg".format(*np.random.randint(0, 9, 3)))),
                             cv2.COLOR_BGR2RGB)
to_face = cv2.cvtColor(cv2.imread(join(data_folder, "img_align_celeba", 
                          "000{}{}{}.jpg".format(*np.random.randint(0, 9, 3)))),
                       cv2.COLOR_BGR2RGB)

In [ ]:
# Load faces with open-cv
from_face = cv2.cvtColor(cv2.imread(join(data_folder, "face_swap", 
                            "carell.png")),
                             cv2.COLOR_BGR2RGB)
to_face = cv2.cvtColor(cv2.imread(join(data_folder, "face_swap", 
                            "steve.jpg")),
                             cv2.COLOR_BGR2RGB)

In [ ]:
plt.imshow(from_face)
plt.show()
plt.imshow(to_face)
plt.show()

# Face Alignment

In [ ]:
# flip destination face if pointing to opposite horizontal direction
landmark_detector.get_face_horizontal_orientation(from_face)

## Landmark Detection

In [ ]:
landmark_detector = utils.LandmarkDetector(join(data_folder, 'shape_predictor_68_face_landmarks.dat'))

In [ ]:
# get face boundary points and containing rectangles
# for both faces
face_boundary_from, rect_from = landmark_detector.get_contour(from_face)
face_boundary_to, rect_to = landmark_detector.get_contour(to_face)

## Find Convex Hull
Get convex hull indexes only of target face, and obtain hull points for both faces using such indexes.

In [ ]:
#hull_idx_from = cv2.convexHull(face_boundary_from, returnPoints = False)
hull_idx_to = cv2.convexHull(face_boundary_to, returnPoints = False)

In [ ]:
#??Do not use directly this cause you might lose correspondence
#between number of points
#hull_from = cv2.convexHull(face_boundary_from, returnPoints = True)
#hull_to = cv2.convexHull(face_boundary_to, returnPoints = True)

In [ ]:
hull_from = np.array([face_boundary_from[hull_idx] for hull_idx in hull_idx_to])
hull_to = np.array([face_boundary_to[hull_idx] for hull_idx in hull_idx_to])

## Delaunay Triangulation

In [ ]:
#triangles_from_idxs = utils.get_triangles_indexes(from_face, rect_from, hull_idx_to, face_boundary_from)
triangles_to_idxs = utils.get_triangles_indexes(to_face, rect_to, hull_idx_to, face_boundary_to)

In [ ]:
triangles_from = utils.clean_triangles((0, 0, from_face.shape[1], from_face.shape[0]), 
                                 utils.delaunay_triangulation(from_face, rect_from, hull_idx_from, face_boundary_from))
triangles_to = utils.clean_triangles((0, 0, to_face.shape[1], to_face.shape[0]), 
                               utils.delaunay_triangulation(to_face, rect_to, hull_idx_to, face_boundary_to))

In [ ]:
print(len(triangles_from))
print(len(triangles_to))

### Plot Triangulation

In [ ]:
import matplotlib.patches as patches

In [ ]:
# Create figure and axes
fig,ax = plt.subplots(1)

# Display the image
ax.imshow(from_face)

for t in triangles_from:
    triangle = patches.Polygon(t, linewidth=1,edgecolor='r',facecolor='none')
    ax.add_patch(triangle)
# Create a Rectangle patch
#rect = patches.Rectangle((50,100),40,30,linewidth=1,edgecolor='r',facecolor='none')

# Add the patch to the Axes
#ax.add_patch(rect)
plt.show()

In [ ]:
# Create figure and axes
fig,ax = plt.subplots(1)
# Display the image
ax.imshow(to_face)

for t in triangles_to:
    triangle = patches.Polygon(t, linewidth=1,edgecolor='r',facecolor='none')
    ax.add_patch(triangle)
# Create a Rectangle patch
#rect = patches.Rectangle((50,100),40,30,linewidth=1,edgecolor='r',facecolor='none')

# Add the patch to the Axes
#ax.add_patch(rect)
plt.show()

In [ ]:
np.array(triangles_to[0])

## Affine warp triangles 

In [ ]:
img_res = utils.image_affine_warp(hull_from,
                      hull_to,
                      triangles_to_idxs, 
                      from_face, 
                      to_face.copy())

In [ ]:
plt.imshow(img_res)
plt.show()

# Seamless Cloning

In [ ]:
output = utils.seamless_cloning(hull_to.reshape(hull_to.shape[0], 2), to_face, img_res)

In [ ]:
plt.imshow(output)
plt.show()